### Install needed packages

In [1]:
!pip install transformers torch pandas accelerate

In [4]:
import pandas as pd
import json
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch
from google.colab import drive

### Mount drive

In [ ]:
drive.mount('/content/drive')

### Load dataset

In [13]:
df = pd.read_csv('/content/drive/MyDrive/app.csv')
print(df.head())  # Inspect the first few rows

                                            sentence  label
0  package file invalid i had my phone on factory...      2
1  iffy nice clean app but sometimes it works and...      2
2                        cool just freezes everytime      2
3  network error! suddenly after downloading an u...      2
4  annoying it let me choose the pictures i want ...      2
